### Download the training CSV from our repo

In [1]:
import os

# URL of the file to be downloaded
url = 'https://github.com/Ensemble-AI-co/Public_datasets/raw/main/Friendly_competition_train.zip'

# Using wget to download the file
os.system(f'wget {url}')

0

### Local Model Preprocessing and Training

In [28]:
df = pd.read_csv('/content/Friendly_competition_train.zip')

<ipython-input-28-e4745f9fa2a1>:1: DtypeWarning: Columns (9) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('/content/Friendly_competition_train.zip')


In [29]:
df.head()

,1,2,3,4,5,6,7,8,9,10,...,162,163,164,165,166,167,168,169,170,Results
0,0.148148,0.033333,0.393939,0.153689,0.145071,0.562333,0.146565,0.144747,0.014625,NaN,...,False,False,False,False,True,False,False,False,False,1.0
1,0.481481,0.033333,0.315632,0.112705,0.132876,0.456500,0.109160,0.132791,0.090201,NaN,...,False,False,False,False,True,False,False,False,False,0.0
2,0.444444,0.233333,0.282028,0.518443,0.683689,0.456500,0.562595,0.684559,0.015614,NaN,...,False,False,False,False,True,False,False,False,False,0.0
3,0.296296,0.233333,0.318332,0.014344,0.016768,0.353500,0.010687,0.016790,0.057424,NaN,...,False,False,False,False,False,True,False,False,False,1.0
4,0.518519,0.400000,0.261026,0.016393,0.023374,0.380500,0.016031,0.023404,0.072221,93.04,...,False,False,False,False,False,False,True,False,False,0.0


#### We observe a very imbalanced dataset

In [24]:
import numpy as np
neg, pos = np.bincount(df['Results'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 22498
    Positive: 2393 (10.64% of total)



In [25]:
!pip install imbalanced-learn

### Create a balanced dataset for training

This is an example local pretraining code, our submission won't need to worry about balancing any data (as its simply for running inference).

In [30]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
import torch
from torch.utils.data import DataLoader, TensorDataset

df = df.drop(['10'], axis=1)
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
undersampler = RandomUnderSampler(random_state=42)
X_train_balanced, y_train_balanced = undersampler.fit_resample(X_train_smote, y_train_smote)
scaler = StandardScaler()
X_train_balanced = scaler.fit_transform(X_train_balanced)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
X_train_balanced = torch.tensor(X_train_balanced, dtype=torch.float32)
y_train_balanced = torch.tensor(y_train_balanced, dtype=torch.long)
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.long)
train_dataset_balanced = TensorDataset(X_train_balanced, y_train_balanced)
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
batch_size = 32
train_loader = DataLoader(dataset=train_dataset_balanced, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset), shuffle=False)

### Verify the training loader is balanced

In [36]:
total_samples = 0
class_counts = {}
for data, labels in train_loader:
    total_samples += len(data)
    for label in labels:
        label = label.item()
        if label in class_counts:
            class_counts[label] += 1
        else:
            class_counts[label] = 1
print("Class Distribution:")
for label, count in class_counts.items():
    percentage = (count / total_samples) * 100
    print(f"Class {label}: {count} samples ({percentage:.2f}% of total)")

Class Distribution:
Class 0: 16086 samples (50.00% of total)
Class 1: 16086 samples (50.00% of total)


### Pytorch Training code

In [39]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import CosineAnnealingLR
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

class BinaryClassifier(nn.Module):
    def __init__(self):
        super(BinaryClassifier, self).__init__()
        self.layer_1 = nn.Linear(169, 128)
        self.dropout_1 = nn.Dropout(0.2)
        self.layer_2 = nn.Linear(128, 64)
        self.dropout_2 = nn.Dropout(0.3)
        self.layer_3 = nn.Linear(64, 32)
        self.dropout_3 = nn.Dropout(0.6)
        self.layer_4 = nn.Linear(32, 16)
        self.dropout_4 = nn.Dropout(0.3)
        self.layer_out = nn.Linear(16, 2)
        self.relu = nn.ReLU()

    def forward(self, inputs):
        x = self.relu(self.layer_1(inputs))
        x = self.dropout_1(x)
        x = self.relu(self.layer_2(x))
        x = self.dropout_2(x)
        x = self.relu(self.layer_3(x))
        x = self.dropout_3(x)
        x = self.relu(self.layer_4(x))
        x = self.dropout_4(x)
        x = self.layer_out(x)
        return x

model = BinaryClassifier()
epochs = 30
weight_decay = 1e-4
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=weight_decay)
scheduler = CosineAnnealingLR(optimizer, T_max=epochs, eta_min=0)

for epoch in range(epochs):
    model.train()
    train_loss = 0.0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * X_batch.size(0)
    train_loss /= len(train_loader.dataset)

    model.eval()
    test_loss = 0.0
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            test_loss += loss.item() * X_batch.size(0)
    test_loss /= len(test_loader.dataset)

    scheduler.step()

    y_true = []
    y_pred = []
    with torch.no_grad():
        for X_batch, y_batch in test_loader:
            outputs = model(X_batch)
            _, predicted = torch.max(outputs, 1)
            y_true.extend(y_batch.tolist())
            y_pred.extend(predicted.tolist())

    f1 = f1_score(y_true, y_pred)
    print(f'Epoch {epoch+1}/{epochs}, Training Loss: {train_loss:.4f}, Test Loss: {test_loss:.4f}, Test F1: {f1:.4f}')


Epoch 1/30, Training Loss: 0.6519, Test Loss: 0.5566, Test F1: 0.3056
Epoch 2/30, Training Loss: 0.5744, Test Loss: 0.5443, Test F1: 0.3133
Epoch 3/30, Training Loss: 0.5439, Test Loss: 0.5511, Test F1: 0.3166
Epoch 4/30, Training Loss: 0.5203, Test Loss: 0.5243, Test F1: 0.3240
Epoch 5/30, Training Loss: 0.5034, Test Loss: 0.5344, Test F1: 0.3283
Epoch 6/30, Training Loss: 0.4888, Test Loss: 0.5289, Test F1: 0.3251
Epoch 7/30, Training Loss: 0.4745, Test Loss: 0.5112, Test F1: 0.3226
Epoch 8/30, Training Loss: 0.4644, Test Loss: 0.5113, Test F1: 0.3335
Epoch 9/30, Training Loss: 0.4535, Test Loss: 0.5129, Test F1: 0.3275
Epoch 10/30, Training Loss: 0.4452, Test Loss: 0.5132, Test F1: 0.3193
Epoch 11/30, Training Loss: 0.4323, Test Loss: 0.5038, Test F1: 0.3179
Epoch 12/30, Training Loss: 0.4223, Test Loss: 0.4856, Test F1: 0.3166
Epoch 13/30, Training Loss: 0.4152, Test Loss: 0.5011, Test F1: 0.3166
Epoch 14/30, Training Loss: 0.4092, Test Loss: 0.5190, Test F1: 0.3104
Epoch 15/30, Tr

### Training observations

Clearly, our model did not train very successfully. Feel free to alter preprocessing, feature engineering, model architecture, and training techniques!

In [42]:
"""USE THIS FUNCTION TO SAVE YOUR MODEL FOR SUBMISSION"""
def save_model(model, model_name: str):
  trained_model_rep = torch.jit.script(model)
  trained_model_rep.save(model_name)

In [41]:
save_model(model, "model.pth")

### preprocess.py
The following cell contains the entire contents of preprocess.py for our example submission. In order to run inference on our model, we only need to drop column 10 and convert everything to tensors.

In [ ]:
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import torch

def create_dataloader_for_test(df, batch_size=64):
   df = df.drop(['10'], axis=1)
   X = df.astype(float)
   X = torch.tensor(X.values, dtype=torch.float32)
   dataset = TensorDataset(X)
   batch_size = 32
   loader = DataLoader(dataset=dataset, batch_size=len(dataset), shuffle=False)
   torch.save(loader, 'dataloader.pth')

### Submission Instructions


Download your trained model.pth from here and submit it on our web UI alongside preprocess.py (see above cell) and requirements.txt. Examples of those 3 files are in the starter repo with this notebook.